In [5]:
import functools
import pprint as pp
import shlex
import os
import inspect
from subprocess import run

In [99]:
def memory_profile(f):
    @functools.wraps(f)
    def inner(*args, **kwargs):
        
        def unwrap_source_code(g):
            sourcelines = inspect.getsourcelines(g)
            for idx, i in enumerate(sourcelines[0]):
                if i.startswith('def'):
                    return "".join(sourcelines[0][idx:])
            
        temp = './temp_ram.py'
        with open(temp, 'w+') as g:
            decorator = '@profile'
            sourcelines = inspect.getsourcelines(f)
            value = f"""import argparse\nimport sys\nimport json\n\n\n{decorator}\n{unwrap_source_code(f)}"""
            g.write(value)
            g.write('\n')
          
            main_exec = f"""if __name__ == "__main__":     
    parser = argparse.ArgumentParser()
    parser.add_argument('-dummy_pos', nargs='*')
    parser.add_argument('--dummy_optional', nargs="?")    
    args = parser.parse_args()

    star_args = args.dummy_pos
    optional_args = args.dummy_optional

    if optional_args:
        optional_args = optional_args.replace("\'", '"')
        optional_args = json.loads(json.loads(json.dumps(optional_args)))
    else:
        optional_args = {{}}  
    
    {f.__name__}(*star_args, **optional_args)
                    """
            g.write(main_exec)
            
        cmd = f'python -m memory_profiler {temp}'
        if args:
            args_split = [f'"{str(i)}"' for i in args]
            second_split =  "-dummy_pos" + " " + " ".join(args_split)
           
            cmd = cmd + " " + second_split
        
        if kwargs:
            cmd = cmd + " " + '--dummy_optional' + " " + " ".join([f'"{ {k : str(v) for k, v in kwargs.items()} }"'])
        
        print(cmd)
        mem_stats = run(cmd, capture_output=True).stdout.decode('utf-8')
        
       
        # TODO: We might want to have also 
        # a `debug` argument, where we don't delete
        # the temporary file.
        
        #os.remove(temp)
        print(mem_stats)
        return f(*args, **kwargs)
    return inner
    
    
    
def timing(f):
    import time
    @functools.wraps(f)
    def inner(*args, **kwargs):
        start = time.perf_counter()
        value = f(*args, **kwargs)
        end = time.perf_counter()
        print(f"Finished {f.__name__} in {end - start:.4f} seconds.")
        return value
    return inner


    


In [98]:
@timing
@memory_profile
def custom(x, y):
    import sklearn
    import matplotlib
    import sklearn


custom(0, 1);



python -m memory_profiler ./temp_ram.py -dummy_pos "0" "1"
Filename: ./temp_ram.py

Line #    Mem usage    Increment   Line Contents
     6   40.078 MiB   40.078 MiB   @profile
     7                             def custom(x, y):
     8   75.863 MiB   35.785 MiB       import sklearn
     9   79.555 MiB    3.691 MiB       import matplotlib
    10   79.555 MiB    0.000 MiB       import sklearn



Finished custom in 4.2404 seconds.
